In [1]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/dictionary/synonym/synonym0.json
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/dictionary/synonym/synonym1.json

In [2]:
import json
from collections import defaultdict

synonyms = defaultdict(list)

files = ['synonym0.json', 'synonym1.json']

for file in files:
    with open(file) as fopen:
        data = json.load(fopen)

    for i in data:
        if not len(i[1]):
            continue
        synonyms[i[0]].extend(i[1])
        
        for r in i[1]:
            synonyms[r].append(i[0])

for k, v in synonyms.items():
    synonyms[k] = list(set(v))

len(synonyms)

10984

In [3]:
with open('../bert/dumping-wiki.txt') as fopen:
    data = list(filter(None, fopen.read().split('\n')))

In [4]:
import re
from tqdm import tqdm

def preprocessing(string):
    string = re.sub(
        'http\S+|www.\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('#') < 0 and i.find('@') < 0]
        ),
    )
    
    chars = ',.()!:\'"/;=-'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub(
        u'[0-9!@#$%^&*()_\-+{}|\~`\'";:?/.>,<]',
        ' ',
        string,
        flags = re.UNICODE,
    )
    string = re.sub(r'[ ]+', ' ', string).strip()
    
    return string.lower().split()

def loop(strings):
    results = []
    for i in tqdm(range(len(strings))):
        p = preprocessing(strings[i])
        if len(p) < 400:
            results.append(p)
    return results

In [5]:
import husein_boilerplate as husein

In [6]:
%%time

data = husein.multiprocessing.multiprocessing(data, loop)

100%|██████████| 107221/107221 [00:05<00:00, 19510.81it/s]


CPU times: user 10.3 s, sys: 2.17 s, total: 12.5 s
Wall time: 17.4 s


In [8]:
counts = []
for string in tqdm(data):
    count = 0
    for no, word in enumerate(string):
        if word in synonyms:
            count += 1
    counts.append(count)

100%|██████████| 1715524/1715524 [00:08<00:00, 210562.01it/s]


In [9]:
import numpy as np

argsort = np.argsort(counts)[::-1][:50000].tolist()

In [10]:
len(data[argsort[0]])

384

In [15]:
import random

def replace(string):
    for no, word in enumerate(string):
        if word in synonyms:
            w = random.choice(synonyms[word])
            string[no] = w
    return ' '.join(string)

In [21]:
def loop(indices):
    results = []
    for index in tqdm(indices):
        string = data[index]
        for _ in range(3):
            results.append((' '.join(string), replace(string)))
    return results

In [22]:
%%time

results = husein.multiprocessing.multiprocessing(argsort, loop)

100%|██████████| 3125/3125 [00:01<00:00, 3099.31it/s]


CPU times: user 335 ms, sys: 1.03 s, total: 1.36 s
Wall time: 2.15 s


In [23]:
results[-2:]

[('piagam ini bersetuju dokumentasi terkimpal dari parti ranah rakyat kelantan darul naim apabila parti pemerintah kawasan sudah membentuk maktab bebola sepak darul naim mendalam menjumpai kebijakan dan mempertingkatkan bahkan harganya perlawanan bebola sepak di ranah ini',
  'program ini membawa bukti padu dari keributan gelanggang negara kelantan darul naim apabila pesta komandan wilayah wilayah telah lukis Akademi tarian sepak darul naim lengkap menentukan kepintaran dan mempertingkatkan pun bayaran pertandingan bola sepak di negara ini'),
 ('program ini membawa bukti padu dari keributan gelanggang negara kelantan darul naim apabila pesta komandan wilayah wilayah telah lukis Akademi tarian sepak darul naim lengkap menentukan kepintaran dan mempertingkatkan pun bayaran pertandingan bola sepak di negara ini',
  'manifesto ini berlanjutan lambang terkimpal dari kehadiran ranah keadaan kelantan darul naim apabila kenduri panglima persekitaran sudah menarik maktab dansa sepak darul naim 

In [25]:
with open('synonyms.json', 'w') as fopen:
    json.dump(results, fopen)